In [ ]:
import firebase_admin
from firebase_admin import credentials, db
import pandas as pd
import joblib
import time

# Inisialisasi Firebase Admin SDK dengan kunci
cred = credentials.Certificate('kebuncabe-6f515-firebase-adminsdk-n54zr-c89d1f51dd.json')

try:
    firebase_admin.get_app()
except ValueError:
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://kebuncabe-6f515-default-rtdb.firebaseio.com/'
    })

# Muat model yang telah dilatih
model = joblib.load('C:\\Users\\ASUS\\Documents\\Machine Learning\\fertilizer_and_water_model_baru.pkl')

# Fungsi untuk memprediksi secara realtime
def predict_fertilizer_realtime():
    while True:
        # Membaca data dari Firebase Realtime Database
        ref = db.reference('sensorData')
        sensor_data = ref.get()

        if not sensor_data:
            print('Tidak ada data sensor di Firebase.')
            time.sleep(60) # Cek lagi dengan jeda 60 detik
            continue

        sensor_records = []

        for key, value in sensor_data.items():
            if isinstance(value, dict) and all(k in value for k in ['moisture', 'ph', 'N', 'P', 'K']):
                sensor_records.append(value)
            else:
                print(f"Data tidak valid untuk kunci {key}. Data diabaikan.")

        if not sensor_records:
            print('Tidak ada data sensor yang valid untuk diproses.')
            time.sleep(60)
            continue

        sensor_df = pd.DataFrame(sensor_records)

        # Debugging: Cek data input yang digunakan untuk prediksi
        print("Data Sensor:")
        print(sensor_df[['moisture', 'ph', 'N', 'P', 'K']])

        # Prediksi jumlah pupuk dan kebutuhan air
        predicted_values = model.predict(sensor_df[['moisture', 'ph', 'N', 'P', 'K']])
        print(f"Predicted values: {predicted_values}")  # Debugging output

        # Konversi dan pembulatan
        predicted_fertilizers_grams = (predicted_values[:, :3] * 0.5).round(2)
        predicted_water_ml = (predicted_values[:, 3]).round(2)
        print(f"Raw predicted water values: {predicted_values[:, 3]}")  # Debugging output
        print(f"Rounded predicted water values: {predicted_water_ml}")  # Debugging output

        # Hitung waktu yang diperlukan untuk air
        water_flow_rate_ml_per_sec = 4.6
        predicted_water_time_sec = (predicted_water_ml / water_flow_rate_ml_per_sec).round(2)

        # Tambah hasil prediksi ke DataFrame
        sensor_df['Penambahan Pupuk Urea (N)'] = predicted_fertilizers_grams[:, 0]
        sensor_df['Penambahan Pupuk SP36 (P)'] = predicted_fertilizers_grams[:, 1]
        sensor_df['Penambahan Pupuk KCL (K)'] = predicted_fertilizers_grams[:, 2]
        sensor_df['Penambahan Air (ml)'] = predicted_water_ml
        sensor_df['Waktu Air (detik)'] = predicted_water_time_sec

        print(sensor_df[['Penambahan Air (ml)']])  # Debugging output

        # Menyimpan prediksi ke Firebase Realtime Database
        predictions = sensor_df[[
            'Penambahan Pupuk Urea (N)', 
            'Penambahan Pupuk SP36 (P)', 
            'Penambahan Pupuk KCL (K)', 
            'Penambahan Air (ml)',
            'Waktu Air (detik)'
        ]].to_dict(orient='records')

        predictions_ref = db.reference('predictions_realtime')
        for i, prediction in enumerate(predictions):
            print(f"Saving prediction {i}: {prediction}")  # Debugging output
            predictions_ref.child(str(i)).set(prediction)

        print('Prediksi untuk data realtime telah disimpan ke Firebase.')

        time.sleep(60)

# Panggil fungsi untuk melakukan prediksi secara realtime
predict_fertilizer_realtime()


Data tidak valid untuk kunci waterLevel. Data diabaikan.
Data Sensor:
   moisture  ph   N   P   K
0        39   4  33  10  37
Predicted values: [[25.  4. 18. 23.]]
Raw predicted water values: [23.]
Rounded predicted water values: [23.]
   Penambahan Air (ml)
0                 23.0
Saving prediction 0: {'Penambahan Pupuk Urea (N)': 12.5, 'Penambahan Pupuk SP36 (P)': 2.0, 'Penambahan Pupuk KCL (K)': 9.0, 'Penambahan Air (ml)': 23.0, 'Waktu Air (detik)': 5.0}
Prediksi untuk data realtime telah disimpan ke Firebase.
Data tidak valid untuk kunci waterLevel. Data diabaikan.
Data Sensor:
   moisture  ph   N   P   K
0        39   4  33  10  37
Predicted values: [[25.  4. 18. 23.]]
Raw predicted water values: [23.]
Rounded predicted water values: [23.]
   Penambahan Air (ml)
0                 23.0
Saving prediction 0: {'Penambahan Pupuk Urea (N)': 12.5, 'Penambahan Pupuk SP36 (P)': 2.0, 'Penambahan Pupuk KCL (K)': 9.0, 'Penambahan Air (ml)': 23.0, 'Waktu Air (detik)': 5.0}
Prediksi untuk data r

Prediksi untuk data realtime telah disimpan ke Firebase.
Data tidak valid untuk kunci waterLevel. Data diabaikan.
Data Sensor:
   moisture  ph   N   P   K
0        39   4  33  10  37
Predicted values: [[25.  4. 18. 23.]]
Raw predicted water values: [23.]
Rounded predicted water values: [23.]
   Penambahan Air (ml)
0                 23.0
Saving prediction 0: {'Penambahan Pupuk Urea (N)': 12.5, 'Penambahan Pupuk SP36 (P)': 2.0, 'Penambahan Pupuk KCL (K)': 9.0, 'Penambahan Air (ml)': 23.0, 'Waktu Air (detik)': 5.0}
Prediksi untuk data realtime telah disimpan ke Firebase.
